In [22]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [23]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNDFiN2E2YmYtNWNkMS00MDBjLThhODEtZDg1NDViNGZlYjJhIiwgInJlY2lwaWVudEtleXMiOiBbIkJjOWJDWVFNb3oyaW1UODlvZjRRdjZ6NXZxbmpTb2VucjJMM2FzdEpKZUhhIl0sICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9hZ2VudDEubG9jYWxob3N0In0=


In [24]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNjQzMmVlZjgtZmE3YS00Y2JhLTg0NzUtMDA4ZGIyY2ZkYWM2IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyI3Y1YyZHRaMlhBaDVFcTVqVnBVeVl6bk5qQnh5YlhjWkRiTHdIcndiclhTZiJdfQ==


In [25]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '6432eef8-fa7a-4cba-8475-008db2cfdac6',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['7cV2dtZ2XAh5Eq5jVpUyYznNjBxybXcZDbLwHrwbrXSf'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [26]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 y9AQqyUrxKUDY4fF7J5a1sWvkW3KB3YgkbUk5eSf5gLsA2DiSUuVqNKgdB3kdvCzDXLBse6MFTiPBwP7zFe47Z1

Public key: 
 D2E1C3wW4VvAjXCFZXMP6jusUoGfEmnd5bFRm7twedFV


**Sending a connection request to acapy**

In [27]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [28]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [29]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [30]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

affd6b1a-8350-4cc2-bcf4-22f2e1c8280d 
 4800931d-4ae9-4e63-b5c2-8da68ad8b2c9


### FEATURE DISCOVERY

In [31]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [11]:
# web.post("/verifiable-services/add", add),
# web.get(
#     "/verifiable-services/send/{connection_id}", getList, allow_head=False
# ),
# web.get("/verifiable-services/{connection_id}", get, allow_head=False),

In [32]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "memxsassdsssdfdeds",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string",
    "data_url": "string"
  }
})
print(add.text)
add = json.loads(add.text)

{"updated_at": "2020-07-23 10:41:17.400377Z", "label": "memxsassdsssdfdeds", "service_schema": {"oca_schema_namespace": "string", "oca_schema_dri": "string"}, "consent_schema": {"data_url": "string", "oca_schema_namespace": "string", "oca_schema_dri": "string"}, "created_at": "2020-07-23 10:41:17.400377Z"}


In [33]:
send = requests.get('http://localhost:8150/verifiable-services/request-list/' + connection[0])
print(send.text)
send = json.loads(send.text)

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/discovery", "@id": "9316f4e9-dd7b-4744-8234-75b20aeb25b0"}


In [34]:
send = requests.get('http://localhost:8150/verifiable-services/get-list/' + connection[0])
send = json.loads(send.text)
pp.pprint(send)

{'connection_id': 'affd6b1a-8350-4cc2-bcf4-22f2e1c8280d',
 'created_at': '2020-07-23 10:41:19.480951Z',
 'services': [{'consent_schema': {'data_url': 'string',
                                  'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'},
               'label': 'memxsassdsssdfdeds',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}}],
 'updated_at': '2020-07-23 10:41:19.480951Z'}


In [35]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/discovery",
)
response = util.sendMessage(message, agents[1])
pp.pprint(response)

{'@id': '5794a7f9-1516-44b6-b873-202a5a58211b',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/discovery-response',
 'services': [{'consent_schema': {'data_url': 'string',
                                  'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'},
               'label': 'memxsassdsssdfdeds',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}}],
 '~thread': {'thid': 'edef1ddb-73b2-4b7b-8edc-28853f6df31c'}}


In [36]:
payload = "some string yes dd"
payload_response = "response yay"
hash_id = hashlib.sha256(payload.encode("UTF-8")).hexdigest()
hash_id_response = hashlib.sha256(payload_response.encode("UTF-8")).hexdigest()
print(payload, hash_id)

some string yes dd b7c34035833ad394aaf721e431ae4a7f6359d66e1c69e8be9ec4c3e1d2e1ea1f


I added returning hash id to add schema, I need to test if it works with both rejecting (not found schema) and accepting schema dound

### Add schema to storage

In [52]:
add = requests.post('http://localhost:8150/schema-storage/add', json={
  "payload": "test"
})
print(add.text)
add = json.loads(add.text)

{"hash_id": "9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08", "payload": "test"}


### SEND Initial Request

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# newDid = util.createDid(agents[0])
# pp.pprint(newDid['result'])

In [ ]:
# util.registerDidOnLedger(agents[0], newDid['result']['did'], newDid['result']['verkey'])

In [ ]:
# util.setPublicDid(agents[0], newDid['result']['did'])

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)